In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler, LabelEncoder
from sklearn.metrics import accuracy_score, confusion_matrix, classification_report
import xgboost as xgb
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Conv1D, Flatten, Dropout
import tkinter as tk

# Load the dataset
df = pd.read_csv('Heart Attack.csv')

# Encoding categorical data
label_encoder = LabelEncoder()
df['class'] = label_encoder.fit_transform(df['class'])  # Assuming 'class' is the target variable

# Splitting the dataset into features and target
X = df.drop(columns=['class'])
y = df['class']

# Splitting the dataset into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Standardizing the features
scaler = StandardScaler()
X_train = scaler.fit_transform(X_train)
X_test = scaler.transform(X_test)

# XGBoost Model
xgb_model = xgb.XGBClassifier(objective='binary:logistic', random_state=42)
xgb_model.fit(X_train, y_train)

# CNN Model
# Reshape data for CNN
X_train_cnn = X_train.reshape(X_train.shape[0], X_train.shape[1], 1)
X_test_cnn = X_test.reshape(X_test.shape[0], X_test.shape[1], 1)

# Define the CNN model
cnn_model = Sequential([
    Conv1D(32, 2, activation='relu', input_shape=(X_train.shape[1], 1)),
    Dropout(0.2),
    Conv1D(64, 2, activation='relu'),
    Flatten(),
    Dense(128, activation='relu'),
    Dropout(0.2),
    Dense(1, activation='sigmoid')
])

# Compile the model
cnn_model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])

# Train the CNN model
history = cnn_model.fit(X_train_cnn, y_train, epochs=50, batch_size=32, validation_data=(X_test_cnn, y_test))

# Function to predict heart disease using both models
def predict_heart_disease(input_data, scaler, xgb_model, cnn_model):
    # Prepare input data for prediction
    input_data = np.array(input_data).reshape(1, -1)
    input_data_scaled = scaler.transform(input_data)

    # XGBoost prediction
    xgb_pred = xgb_model.predict(input_data_scaled)
    xgb_result = xgb_pred[0]

    # Prepare input data for CNN prediction
    input_data_cnn = input_data_scaled.reshape(input_data_scaled.shape[0], input_data_scaled.shape[1], 1)

    # CNN prediction
    cnn_pred = (cnn_model.predict(input_data_cnn) >= 0.5).astype(int).flatten()
    cnn_result = cnn_pred[0]

    # Hybrid prediction using majority voting
    hybrid_result = 1 if xgb_result + cnn_result > 1 else 0

    return hybrid_result

# Function to compute hybrid model accuracy
def compute_hybrid_accuracy(X_test, X_test_cnn, y_test, xgb_model, cnn_model):
    xgb_pred = xgb_model.predict(X_test)
    cnn_pred = (cnn_model.predict(X_test_cnn) >= 0.5).astype(int).flatten()

    # Combine predictions using majority voting
    hybrid_pred = []
    for xgb, cnn in zip(xgb_pred, cnn_pred):
        if xgb + cnn > 1:
            hybrid_pred.append(1)
        else:
            hybrid_pred.append(0)

    hybrid_accuracy = accuracy_score(y_test, hybrid_pred)
    hybrid_cm = confusion_matrix(y_test, hybrid_pred)
    hybrid_cr = classification_report(y_test, hybrid_pred)

    return hybrid_accuracy, hybrid_cm, hybrid_cr

# Create a tkinter GUI
def predict():
    # Clear previous results and error messages
    result_label.config(text="")
    error_text.set("")
    
    error_msgs = []

    try:
        # Get input data from entry widgets
        input_data = [
            int(entries[0].get()),  # Age
            int(entries[1].get()),  # Gender
            int(entries[2].get()),  # Impulse
            int(entries[3].get()),  # High Pressure
            int(entries[4].get()),  # Low Pressure
            float(entries[5].get()),  # Glucose
            float(entries[6].get()),  # KCM
            float(entries[7].get())   # Trophonin
        ]
        
        # Validate input data ranges
        if input_data[0] <= 0 or input_data[0] > 120:
            error_msgs.append("Age should be between 1 and 120.")
        if input_data[1] not in [0, 1]:
            error_msgs.append("Gender should be 0 (Female) or 1 (Male).")
        # Add validations for other inputs as needed
        if input_data[2] <= 0:
            error_msgs.append("Impulse should be greater than 0.")
        if input_data[3] <= 0:
            error_msgs.append("High Pressure should be greater than 0.")
        if input_data[4] <= 0:
            error_msgs.append("Low Pressure should be greater than 0.")
        if input_data[5] <= 0:
            error_msgs.append("Glucose should be greater than 0.")
        if input_data[6] <= 0:
            error_msgs.append("KCM should be greater than 0.")
        if input_data[7] <= 0:
            error_msgs.append("Trophonin should be greater than 0.")
        
        if error_msgs:
            error_text.set("\n".join(error_msgs))
            return
        
        # Perform prediction using the hybrid model
        hybrid_result = predict_heart_disease(input_data, scaler, xgb_model, cnn_model)
        
        # Display results beside the interface
        result_text = f"Hybrid Model Prediction: {'Heart Disease' if hybrid_result == 1 else 'No Heart Disease'}"
        result_label.config(text=result_text)
    except ValueError as ve:
        error_text.set(f"Invalid input: {str(ve)}.")
    except Exception as e:
        error_text.set(f"An error occurred: {str(e)}")

# Function to show model evaluation
def show_evaluation():
    # Clear previous evaluation results
    eval_label.config(text="")
    
    # XGBoost evaluation
    y_pred_xgb = xgb_model.predict(X_test)
    xgb_accuracy = accuracy_score(y_test, y_pred_xgb)
    xgb_cm = confusion_matrix(y_test, y_pred_xgb)
    xgb_cr = classification_report(y_test, y_pred_xgb)
    
    # CNN evaluation
    y_pred_cnn = (cnn_model.predict(X_test_cnn) >= 0.5).astype(int).flatten()
    cnn_accuracy = accuracy_score(y_test, y_pred_cnn)
    cnn_cm = confusion_matrix(y_test, y_pred_cnn)
    cnn_cr = classification_report(y_test, y_pred_cnn)

    # Hybrid model evaluation
    hybrid_accuracy, hybrid_cm, hybrid_cr = compute_hybrid_accuracy(X_test, X_test_cnn, y_test, xgb_model, cnn_model)
    
    # Plot confusion matrices
    fig, axs = plt.subplots(1, 3, figsize=(18, 6))
    sns.heatmap(xgb_cm, annot=True, fmt='d', ax=axs[0], cmap='Blues')
    axs[0].set_title('XGBoost Confusion Matrix')
    axs[0].set_xlabel('Predicted')
    axs[0].set_ylabel('Actual')

    sns.heatmap(cnn_cm, annot=True, fmt='d', ax=axs[1], cmap='Blues')
    axs[1].set_title('CNN Confusion Matrix')
    axs[1].set_xlabel('Predicted')
    axs[1].set_ylabel('Actual')

    sns.heatmap(hybrid_cm, annot=True, fmt='d', ax=axs[2], cmap='Blues')
    axs[2].set_title('Hybrid Model Confusion Matrix')
    axs[2].set_xlabel('Predicted')
    axs[2].set_ylabel('Actual')

    plt.show()
    
    # Print classification reports
    print("XGBoost Classification Report:")
    print(xgb_cr)
    print("CNN Classification Report:")
    print(cnn_cr)
    print("Hybrid Model Classification Report:")
    print(hybrid_cr)

    # Show accuracy scores beside the interface
    accuracy_text = f"XGBoost Accuracy: {xgb_accuracy:.2f}\nCNN Accuracy: {cnn_accuracy:.2f}\nHybrid Model Accuracy: {hybrid_accuracy:.2f}"
    eval_label.config(text=accuracy_text)

# Create the main window
window = tk.Tk()
window.title("Heart Disease Prediction")
window.configure(bg="#f0f0f0")  # Set background color

# Define colors for the labels and entries
label_bg_color = "#e6f7ff"  # Light blue
label_fg_color = "#003366"  # Dark blue
entry_bg_color = "#ffe6e6"  # Light red
entry_fg_color = "#660000"  # Dark red

# Create labels and entry widgets for input features
labels = [
    'Age (1-120):', 'Gender (0 = Female, 1 = Male):', 'Impulse(30-200 bpm):', 
    'High Pressure(80-220 mm hg):', 'Low Pressure(50-130 mm hg):', 'Glucose(50-300 mg/dl):', 'KCM(0-400):', 'Trophonin(0-10):'
]
entries = []
error_labels = []

for i, label_text in enumerate(labels):
    label = tk.Label(window, text=label_text, bg=label_bg_color, fg=label_fg_color, font=('Arial', 10))
    label.grid(row=i, column=0, padx=10, pady=5)
    entry = tk.Entry(window, bg=entry_bg_color, fg=entry_fg_color, font=('Arial', 10))
    entry.grid(row=i, column=1, padx=10, pady=5)
    entries.append(entry)
    error_label = tk.Label(window, text="", bg="#f0f0f0", fg="red", font=('Arial', 10))
    error_label.grid(row=i+len(labels), columnspan=2)
    error_labels.append(error_label)

# Create a button to predict heart disease
predict_button = tk.Button(window, text="Predict", bg="#4CAF50", fg="white", font=('Arial', 12), command=predict)
predict_button.grid(row=len(labels)+1, column=0, columnspan=2, pady=10)

# Create a label to display prediction results
result_label = tk.Label(window, text="", bg="#f0f0f0", fg="#003366", font=('Arial', 14, 'bold'))
result_label.grid(row=len(labels)+2, column=0, columnspan=2, pady=10)

# Create a label to display evaluation and accuracy scores
eval_label = tk.Label(window, text="", bg="#f0f0f0", fg="#003366", font=('Arial', 12))
eval_label.grid(row=len(labels)+3, column=0, columnspan=2, pady=10)

# Variable to hold error messages
error_text = tk.StringVar()

# Function to show model evaluation
evaluate_button = tk.Button(window, text="Evaluate Models", bg="#008CBA", fg="white", font=('Arial', 12), command=show_evaluation)
evaluate_button.grid(row=len(labels)+4, column=0, columnspan=2, pady=10)

# Run the tkinter main loop
window.mainloop()
